In [1]:
import warnings
warnings.filterwarnings("ignore")
import os
from recommender_tool import FashionOutfitGenerator
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder, PromptTemplate
from langchain.chains import ConversationalRetrievalChain,LLMChain, ConversationChain
from langchain.chains.question_answering import load_qa_chain
from config import *

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [10]:
memory = ConversationBufferMemory(memory_key="chat_history",return_messages=True)
llm = ChatOpenAI(temperature=0,openai_api_key=OPENAI_API_KEY)

In [11]:
template = """Given the following chat history between a User and AI,
Rephrase the user input to be a standalone input or end the conversation if it seems like it's done.
Chat History:\"""
{chat_history}
\"""
User input: \"""
{input}
\"""
Standalone input:"""
 
condense_question_prompt = PromptTemplate(template=template,input_variables=["chat_history", "input"])

In [12]:
template = """
You are a friendly, conversational clothing shopping assistant.
Use the following context to recommend the outfit with product names, descriptions, and keywords to show the shopper whats available, 
help find what they want, and answer any questions. Be descriptive in your answers.
Context:
\"""
{chat_history}
\"""
Question:\"
{input}
\"""
Helpful answer:
"""

prompt = PromptTemplate.from_template(template=template)

In [13]:
query_refiner = LLMChain(llm=llm,
                         prompt=condense_question_prompt,
                         verbose=False)
main_llm_chain = LLMChain(llm=llm,
                          prompt=prompt,
                          verbose=False,memory=memory)
                          

In [14]:
def func(inp):
    refined_query = query_refiner.run(input=inp,chat_history=memory.buffer_as_messages)
    print(refined_query)
    response = main_llm_chain.run(input=refined_query)
    print(response)

In [15]:
func("Hi")

Hi
Hello! Welcome to our clothing store. How can I assist you today? Are you looking for a specific type of clothing or do you need help with styling?


In [16]:
func("Suggest some outfits for me. I am an investment banker")

Suggest some outfits for an investment banker.
Of course! For an investment banker, it's important to dress professionally and exude confidence. Here's a stylish outfit suggestion:

1. Suit: A well-tailored suit is a must-have for any investment banker. I recommend the "Classic Navy Wool Suit" from our premium collection. It features a slim-fit design, a two-button jacket, and flat-front trousers. This suit is made from high-quality wool fabric, ensuring both comfort and durability.

2. Dress Shirt: Pair the suit with a crisp white dress shirt for a polished look. Our "Slim-Fit White Dress Shirt" is made from 100% cotton, offering a comfortable and breathable fit. It features a spread collar and French cuffs, adding a touch of sophistication.

3. Tie: Complete the ensemble with a sleek tie. Our "Navy Silk Tie with Subtle Pattern" is a perfect choice. It adds a subtle pop of color and texture to the outfit, while still maintaining a professional appearance.

4. Shoes: Opt for a pair of 

In [17]:
func("Suggest something else instead of the Navy Silk Tie.")

Can you suggest an alternative to the Navy Silk Tie?
Certainly! If you're looking for an alternative to the Navy Silk Tie, I recommend our "Burgundy Silk Tie with Geometric Pattern". This tie adds a touch of color and visual interest to your outfit while still maintaining a professional appearance. It is made from high-quality silk and features a subtle geometric pattern that adds a modern twist. Pair it with the Classic Navy Wool Suit, Slim-Fit White Dress Shirt, and Black Leather Oxford Shoes for a stylish and confident look. Let me know if there's anything else I can assist you with!


In [18]:
func("What is the pattern for this tie ?")

What is the pattern for the "Burgundy Silk Tie with Geometric Pattern"?
The pattern for the "Burgundy Silk Tie with Geometric Pattern" is a subtle geometric pattern. This pattern adds a modern twist to the tie and adds visual interest to your outfit. It is a great alternative to the Navy Silk Tie and pairs well with the Classic Navy Wool Suit, Slim-Fit White Dress Shirt, and Black Leather Oxford Shoes. Let me know if there's anything else I can assist you with!


In [19]:
func("Thanks.")

Thanks for your assistance.
You're welcome! I'm glad I could assist you. If you have any more questions or need further assistance, feel free to ask. Happy shopping!


In [20]:
func("Oh, by the way, what was tie's color?")

What was the color of the tie?
The color of the tie is burgundy.


In [21]:
func("And pattern ?")

What is the pattern for the "Burgundy Silk Tie with Geometric Pattern"?


Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-T5FOoUgGaJmatZrIQDka0Zlb on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..


The pattern for the "Burgundy Silk Tie with Geometric Pattern" is a subtle geometric pattern. This pattern adds a modern twist to the tie and adds visual interest to your outfit. It is a great alternative to the Navy Silk Tie and pairs well with the Classic Navy Wool Suit, Slim-Fit White Dress Shirt, and Black Leather Oxford Shoes. Let me know if there's anything else I can assist you with!


In [22]:
print(memory.buffer_as_str)

Human: Hi
AI: Hello! Welcome to our clothing store. How can I assist you today? Are you looking for a specific type of clothing or do you need help with styling?
Human: Suggest some outfits for an investment banker.
AI: Of course! For an investment banker, it's important to dress professionally and exude confidence. Here's a stylish outfit suggestion:

1. Suit: A well-tailored suit is a must-have for any investment banker. I recommend the "Classic Navy Wool Suit" from our premium collection. It features a slim-fit design, a two-button jacket, and flat-front trousers. This suit is made from high-quality wool fabric, ensuring both comfort and durability.

2. Dress Shirt: Pair the suit with a crisp white dress shirt for a polished look. Our "Slim-Fit White Dress Shirt" is made from 100% cotton, offering a comfortable and breathable fit. It features a spread collar and French cuffs, adding a touch of sophistication.

3. Tie: Complete the ensemble with a sleek tie. Our "Navy Silk Tie with S